In [7]:
using EnvelopeApproximation
using EnvelopeApproximation.BubbleBasics
using EnvelopeApproximation.StressEnergyTensor
import LinearAlgebra: norm
using Plots



# Setting up the bubbles

In [ ]:
R = 2.
c = Point3(0., 0., 3.)
bubbles = Bubbles([Bubble(c, R)])

# Setting up the Ks

In [ ]:
k_0 = 2π / R
ks = LinRange(k_0 / 10, k_0 * 10, 1000)
kvecs = (x -> Vec3(0., 0., x)).(ks)


# Computing Analytically

In [ ]:
ΔV = 1.
analytic_T_ii = @. (cos(kvecs ⋅ (c.coordinates, ))) * ((ΔV * 4π / 3) * (R ^ 3)) * sin(ks * R) / (ks * R)  
plot(ks, analytic_T_ii .|> real)

# Computing Numerically

In [ ]:
import EnvelopeApproximation.StressEnergyTensorFFT: TensorDirection, surface_integral, SphericalXX, SphericalYY, SphericalZZ, SphericalTrace
tensor_directions = (SphericalTrace(), SphericalXX(), SphericalYY(), SphericalZZ())
n = 10
@time numerical_T = surface_integral(ks, 0.:0., 0.:0., 100, bubbles, tensor_directions, 10, 10, ΔV)[:, 1, 1, :]

In [ ]:

numerical_ii_integral = numerical_T |> x -> x[:, 1] |> real

In [ ]:
plot(ks, analytic_T_ii, label="analytic")
plot!(ks, numerical_ii_integral, label="numeric", alpha=0.7)

In [ ]:
plot(ks, numerical_ii_integral - analytic_T_ii)

# Study Degree of Realness 

In [ ]:
plot(ks, numerical_ii_integral .|> imag)

# Study tensor directions

In [ ]:
ks

In [ ]:
analytical_T_xx = @. cos(kvecs ⋅ (c.coordinates, )) * 4π / 3 * ΔV  / (ks ^ 3) * (sin(ks * R) - (ks * R) * cos(ks * R))

In [ ]:
plot(ks, numerical_T[:, 2] .|> real, label="(:x, :x)")
plot!(ks, numerical_T[:, 3] .|> real, label="(:y, :y)", alpha=0.5)
plot!(ks, analytical_T_xx .|> real, label="analytic", alpha = 0.5)

In [ ]:
plot(ks, (numerical_T[:, 2] .|> real) - (numerical_T[:, 3] .|> real), label="difference")


## Z, Z

In [ ]:
analytic_zz_integral = @. (cos(kvecs ⋅ (c.coordinates, ))) * (4π / 3 * ΔV * (1 / (ks ^ 3)) * (2 * ks * R * cos(ks * R) + (R^2 * ks^2 - 2) * sin(ks * R)))

In [ ]:
plot(ks, analytic_zz_integral)

In [ ]:
numerical_zz_integral = numerical_T[:, 4] |> real

In [ ]:
plot(ks, analytic_zz_integral, label="analytic")
plot!(ks, numerical_zz_integral, label="numeric")

In [ ]:
plot(ks, (analytic_zz_integral - numerical_zz_integral))